#Do Elite Reviewers Tend to Review the Unreviewed?

Our question asks if elite reviewers tend to review businesses if they do not have many reviews. We can't get inside the heads of the elite reviewers, but as businesses have more reviews, are more of the reviews written by users who are or have been elite, or are more of the reviews written by non-elite users? In other words, do places with higher review counts, meaning that it is popular, tend to be targetted by elite reviewers or is it indeed the opposite where they tend to want to review an unpopular business with less reviews. Because we do not know how an elite reviewers thinks and decide which businesses to review, we will need to draw our insight from somewhere else. To approach this question, we have dive into the Yelp dataset and take a look at some of fields we need for our analysis. We will be taking these fields from three different files: business.json, user.json, review.json.

#Business Data
To start off, business data from the business JSON file is needed in order to match review count and reviews to each business. We will be loading this data into a dataframe to contain the columns that we select. The `business_id` is needed to uniquely identify each business, and the `review_count` to get the amount of reviews that each business has.

In [ ]:
df_business = spark.read.json("/yelp/business.bz2").select("business_id", "review_count").cache()
df_business.show()

+--------------------+------------+
 business_id|review_count|
+--------------------+------------+
f9NumwFMBDn751xgF...| 36|
Yzvjg0SayhoZgCljU...| 4|
XNoUzKckATkOD1hP6...| 5|
6OAZjbxqM5ol29BuH...| 3|
51M2Kk903DFYI6gnB...| 26|
cKyLV5oWZJ2NudWgq...| 38|
oiAlXZPIFm2nBCt0D...| 81|
ScYkbYNkDgCneBrD9...| 18|
pQeaRpvuhoEqudo3u...| 5|
EosRKXIGeSWFYWwpk...| 16|
MbZMmwo-eL0Jnm_Yb...| 3|
7Dv4_HAxsxvadEsT5...| 5|
M_guz7Dj7hX0evS67...| 6|
JjJs3o60uQCfctDjs...| 10|
kOICO53wbOiOJcKuC...| 5|
rqcOZePlVvJP9Etzl...| 5|
uZuh51lXu7tsrC8RA...| 32|
nIEhsGbw0vJuYl05b...| 7|
edQoeeBFUTYGwnUSE...| 10|
Vwo64kNYDjKi98gUU...| 16|
+--------------------+------------+
only showing top 20 rows

The next cell is to condense any reviews past 100 to be equal to 100. This way we don't have a long tail for the histogram. The max # of reviews can be adjusted but we condense them all because businesses with more than 100 reviews accounts for a very small % of total reviews, so adding anything from 100 to 3000 reviews to just be the same thing gives a better idea of the trend.

In [ ]:
df_business.createOrReplaceTempView("business")
df_business_summary = spark.sql("""
SELECT IF(review_count>100, 100, review_count) as review_count, business_id
FROM business
""")

#Taking a Look at our Data

After manipulating our data, we wanted to see how our data is distributed in the form of a histogram. Creating this visualization helped us gain a better understanding of how the data is laid out so that we can determine the appropriate minimum and maximum that captures most of the businesses in the dataset. The histogram was created using the field `review_count` and density was automatically calculated to describe the percentage of businesses in relation to the whole. As you can see, the distribution is skewed to the right with the review counts larger than 100 aggregated to the value 100. We set the max count at 100 since businesses that have more than 100 reviews only makes up for 8% of the data. The reviews past the first 100 will additionally be filtered out later on because each additional review after the first 100 provides less information about the overall trend of the information while also increasing the amount of data that would need to be processed.

In [ ]:
df_business_summary.display()

review_count,business_id
36,f9NumwFMBDn751xgFiRbNA
4,Yzvjg0SayhoZgCljUJRF9Q
5,XNoUzKckATkOD1hP6vghZg
3,6OAZjbxqM5ol29BuHsil3w
26,51M2Kk903DFYI6gnB5I6SQ
38,cKyLV5oWZJ2NudWgqs8VZw
81,oiAlXZPIFm2nBCt0DHLu_Q
18,ScYkbYNkDgCneBrD9vqhCQ
5,pQeaRpvuhoEqudo3uymHIQ
16,EosRKXIGeSWFYWwpkbhNnA


# Profiling Number of Records Based on Minimum Review account

This is the profiling of each minimum that was set to see what amount of reviews should be set as the minimum amount of records for the histogram to be useful. The table shown below details the number of businesses that have a certain amount of reviews

| Min | # of records |
| ------- |
| >=20  |  66117  | 
| >=30  |  48922  | 
| >=40 | 38765 | 
| >=50 | 31942 | 
| >=60 | 26922 | 

Based on this table we chose our minimum to be 30 reviews which gives us 48,922 records to work with. We decided on this number because selecting a minimum lower than that would skew our results for the proption of elite and non-elite reviewers. For instance one elite review out of 20 reviews already accounts for 5% of the reviews. Selecting a minimum that was too high would result in lots of busineses being filtered out which we do not want. We want to account for as many businesses as possible. If we selected our minimum as 40 compared to 30, we are losing roughly 10,000 business that we could use in our data.

In [ ]:
df_business_summary = df_business_summary.filter("review_count>=30")
df_business_summary.count()

Out[4]: 48922

#Users Data
User data is needed in order to decipher if users are elite/non-elite and how many years they have been actively classified as an elite user. In order to do so we will be importing the `user_id` field to determine the unique users that are creating reviews, and we will be importing the `elite` field to see what years they've been elite. This allows for being able to see the difference between when yelp reviews are categorized as elite per the year that they reviewed as elite reviewers, or whether all of the reviews from those who have been elite reviewers are considered to be elite reviews.

In [ ]:
df_users = spark.read.format('parquet').table("users_table").select("user_id", "elite").cache()
df_users.show()

+--------------------+-----+
 user_id|elite|
+--------------------+-----+
Qw_H_0xUHJN1eybBm...| |
E3K2qQ0bkWrSyiikh...| |
Ba_mM1i4OAuZ57MVY...| |
GRujMvFN83-IBQAwT...| |
b4YX1azbMwgV5VLu3...| |
pbenW4yj10LaugmC7...| |
x2fKuZV85FvrSxH-o...| |
Ru7u8IMunZswWbQZD...| |
QB1Ds4_-Dd48ho5pM...| |
cfFS07Gzi5-xSWkQj...| |
uxCTAXR2b5j_UY3PY...| |
x0OHPeQSueJYBEde1...| |
uIPRJrQ8RI1iRG0f5...| |
Exps1-qakisEggqNs...| |
W3AFSt-zVgBl-8kCV...| |
Kv2s1OlVuq0wYlnc_...| |
LjWmAmykgZYHXDa_Q...| |
lHMgyE4hF18kROw-x...| |
gjAYV29Ln0EnNEVTT...| |
jCy-JsEKBaU4wSKoY...| |
+--------------------+-----+
only showing top 20 rows

#Reviews Data
To show when elite users are submitting their reviews, we will extract the `date` that the reviews were conducted. This data will allows us to sort between elite and non-elite users. To match the reviews with each uniquely identified bussiness, the `business_id` will be imported, too. We will match the users to each review by importing the `user_id`.

In [ ]:
df_reviews = spark.read.format('parquet').table("reviews_without_text_table").select("business_id", "user_id", "date").cache()
df_reviews.show()
df_reviews.count()

+--------------------+--------------------+-------------------+
 business_id| user_id| date|
+--------------------+--------------------+-------------------+
3chd-CjwQz7Pap_MQ...|c4Y3EX5aXYixkyVD6...|2019-09-04 19:16:55|
XV45a3fz1d7SRoash...|fHTAUYCd5HuboQp6H...|2016-03-01 21:46:00|
BxcLVgA6BndKDJol0...|ecXDhY91U-ORKk_U7...|2019-11-18 07:13:06|
nSPh4_BKX3Fch0TOa...|ZKpETYqLymDFL76hw...|2019-08-15 16:30:53|
KrcmoxVBX9o3OxFog...|zcp389V72h0DKl69X...|2019-10-24 20:37:02|
ewn93D_Q6bnVosJtw...|xbksPRSmtsmDNiR1V...|2019-11-19 15:33:33|
fKjeOlJTQ1W0cSDRY...|dG-J0IG122Nwqp0Co...|2019-10-21 00:15:38|
b7xistZfcbSO6_0Mg...|O4ADr-eEYkP1OKQUx...|2017-12-22 07:26:39|
6pM2YxQH8dQYs07_C...|LeSXKUJeFvX2aSvfP...|2019-04-15 14:20:28|
iVjeR9z1xVt2ZpxVU...|_fX--5YqPXMB4b7q2...|2019-11-16 22:29:28|
t0edCFMUYmFSN_eXT...|HhfNSpdHGSGiuyi1a...|2019-08-13 03:02:45|
TumOl-iIDkZFRM1k9...|YKiRYAlL7SEV5P29p...|2019-03-15 20:26:47|
ndjiWXoH1pPMwEBhR...|Zt_z6uOk0fPXjsFvC...|2019-11-19 14:47:04|
z8lagvcw1Y4Pz4dVN...|y0HCy6GpDFnsQXhCG...|2019-11-24 18:17:55|
0OZckGHdpbhMMy8ex...|SB7Ow6Y52kNcHdecd...|2018-04-06 03:23:10|
d10IxZPirVJlOSpdR...|0SrL97xFh-mVMOxUF...|2017-03-10 03:47:23|
J3EW9_IQQqkp4nN3q...|yZ3Zm1fBfqlBWhFik...|2015-04-06 00:50:59|
E9HkUYYO_-5vKjvBw...|hRzmJCPmV1_ggM5jS...|2019-10-26 04:00:14|
lNBhwf_3c0QBqBlsY...|HS3y3GltOH_pJ37-E...|2019-11-27 14:34:34|
Kp4i7on7rxFRkWceK...|R-qaBztGNZoxEbKet...|2015-04-27 07:48:39|
+--------------------+--------------------+-------------------+
only showing top 20 rows

Out[6]: 8021122

In [ ]:
#This cell creates temp views to prepare for the next cell's join function
df_users.createOrReplaceTempView("users")
df_reviews.createOrReplaceTempView("reviews")

### Identifying Elite Status 
The query below will select: `elite_years`, `user_id`, `bussiness_id`, and `date`. The data extracted will help us visualize each user's elite_status by year and the date that they conducted the review. We are using the `SPLIT` function to make the elite years an array so we can filter by elite-users later on. The data will match the review to a user via common `user_id` so that we can later determine whether or not each individual review is considered an elite review or not.

In [ ]:
#This cell joins the reviews to the users through the left outer join and splits the elite_years into an array
df_user_reviews = spark.sql("""
SELECT SPLIT(u.elite,'\s*,\s*') as elite_years, U.user_id, R.business_id, R.date
FROM reviews AS R INNER JOIN users AS u
ON R.user_id = U.user_id
""")
df_user_reviews.show()

+--------------------+--------------------+--------------------+-------------------+
 elite_years| user_id| business_id| date|
+--------------------+--------------------+--------------------+-------------------+
 []|c4Y3EX5aXYixkyVD6...|3chd-CjwQz7Pap_MQ...|2019-09-04 19:16:55|
 []|fHTAUYCd5HuboQp6H...|XV45a3fz1d7SRoash...|2016-03-01 21:46:00|
 [2018]|ecXDhY91U-ORKk_U7...|BxcLVgA6BndKDJol0...|2019-11-18 07:13:06|
 []|ZKpETYqLymDFL76hw...|nSPh4_BKX3Fch0TOa...|2019-08-15 16:30:53|
 [2018]|zcp389V72h0DKl69X...|KrcmoxVBX9o3OxFog...|2019-10-24 20:37:02|
[2015, 2016, 2017...|xbksPRSmtsmDNiR1V...|ewn93D_Q6bnVosJtw...|2019-11-19 15:33:33|
 []|dG-J0IG122Nwqp0Co...|fKjeOlJTQ1W0cSDRY...|2019-10-21 00:15:38|
 []|O4ADr-eEYkP1OKQUx...|b7xistZfcbSO6_0Mg...|2017-12-22 07:26:39|
 []|LeSXKUJeFvX2aSvfP...|6pM2YxQH8dQYs07_C...|2019-04-15 14:20:28|
 []|_fX--5YqPXMB4b7q2...|iVjeR9z1xVt2ZpxVU...|2019-11-16 22:29:28|
[2012, 2013, 2014...|HhfNSpdHGSGiuyi1a...|t0edCFMUYmFSN_eXT...|2019-08-13 03:02:45|
 []|YKiRYAlL7SEV5P29p...|TumOl-iIDkZFRM1k9...|2019-03-15 20:26:47|
 [2018]|Zt_z6uOk0fPXjsFvC...|ndjiWXoH1pPMwEBhR...|2019-11-19 14:47:04|
 []|y0HCy6GpDFnsQXhCG...|z8lagvcw1Y4Pz4dVN...|2019-11-24 18:17:55|
 []|SB7Ow6Y52kNcHdecd...|0OZckGHdpbhMMy8ex...|2018-04-06 03:23:10|
 []|0SrL97xFh-mVMOxUF...|d10IxZPirVJlOSpdR...|2017-03-10 03:47:23|
 []|yZ3Zm1fBfqlBWhFik...|J3EW9_IQQqkp4nN3q...|2015-04-06 00:50:59|
 [2012, 2013, 2014]|hRzmJCPmV1_ggM5jS...|E9HkUYYO_-5vKjvBw...|2019-10-26 04:00:14|
 []|HS3y3GltOH_pJ37-E...|lNBhwf_3c0QBqBlsY...|2019-11-27 14:34:34|
 []|R-qaBztGNZoxEbKet...|Kp4i7on7rxFRkWceK...|2015-04-27 07:48:39|
+--------------------+--------------------+--------------------+-------------------+
only showing top 20 rows

In [ ]:
#Unpersist dataframes that are no longer used to free up memory
df_users.unpersist()
df_reviews.unpersist()

Out[9]: DataFrame[business_id: string, user_id: string, date: string]

#Identifying Elite Status by Matching Year
The dateframe below will check if the year of the review matches any of year that the user was an elite member. The `is_elite1` column will display `TRUE` if their is any years in the array of `elite_years`, and `FALSE` is no years are present in the array. This checks whether the user was an elite user in any given year.

In [ ]:
df_user_reviews.createOrReplaceTempView("user_reviews")
df_user_elite_1 = spark.sql("""
SELECT elite_years, user_id, business_id, date, IF(ARRAY_CONTAINS(elite_years, CAST(YEAR(date) AS STRING)), True, False) as is_elite1
FROM user_reviews
""")
df_user_elite_1.show(100)

+--------------------+--------------------+--------------------+-------------------+---------+
 elite_years| user_id| business_id| date|is_elite1|
+--------------------+--------------------+--------------------+-------------------+---------+
 []|---RfKzBwQ8t3wu-L...|Z3ZSar8IVAR2qIupq...|2018-01-13 04:09:32| false|
 []|--5Dnq-kuMq5bqvlg...|YYD8tl3ODut_D_sqd...|2014-07-23 02:57:42| false|
 []|--DvZltAxZ1sjFn6m...|2t-a8qkgoRiiohThD...|2019-02-02 22:51:46| false|
 []|--E5NqaDhTIANG0mX...|JY4kqpElqfflV51PG...|2019-05-22 20:03:46| false|
 []|--FoFf79nPpKASxw0...|qNYLt8zizn--wEpT1...|2019-05-06 18:29:22| false|
 []|--HeZYR8uZadRXLAQ...|_ZfjpSEO5ntk-1hbn...|2016-04-02 18:03:34| false|
 []|--HeZYR8uZadRXLAQ...|ebTvBxSStI9Vf5Tpu...|2016-09-13 13:21:46| false|
 []|--I4wRDhmM2J2VLzK...|JmI9nslLD7KZqRr__...|2014-09-14 23:33:43| false|
 []|--Oi-hOzz5vCLn9l6...|KligxMWsVPp7Jr_YZ...|2015-04-17 16:32:28| false|
 []|--PJ1FuEoTEo-3Cxf...|RESDUcs7fIiihp38-...|2016-08-11 20:51:16| false|
 []|--PJ1FuEoTEo-3Cxf...|2weQS-RnoOBhb1KsH...|2017-01-08 21:22:47| false|
 []|--RYvmB6UYRyZQqXk...|ZMbY3xmTJ3EB2f1ZF...|2011-08-29 16:41:33| false|
 []|--RYvmB6UYRyZQqXk...|MsUI2BgJjptqa1YEA...|2013-11-02 15:24:29| false|
 []|--RYvmB6UYRyZQqXk...|i7WdEjPrZkzo7whKd...|2013-01-10 02:50:28| false|
 []|--XsqVlXNtt20d5zS...|wk6aHP-vxv9dhmmJV...|2014-07-01 04:12:22| false|
 []|--XsqVlXNtt20d5zS...|3enxXjE0vtVaZ-un4...|2014-06-14 01:20:16| false|
 []|--XsqVlXNtt20d5zS...|zGXO7B16FXxjc_P0r...|2014-07-01 04:08:19| false|
 []|--YhC3mgAU7iexygW...|Xcal78l5dziYYUOfL...|2016-03-15 23:16:11| false|
 []|--agAy0vRYwG6WqbI...|ATeOjZzE3VOI-RW7F...|2013-11-26 22:46:06| false|
 []|--agAy0vRYwG6WqbI...|chmsA4G55YHyEMsVp...|2014-12-25 23:30:10| false|
 []|--bp0mKBU1gk9FUTS...|as_fyujgsdwtV56n3...|2016-04-07 02:07:56| false|
 []|--bp0mKBU1gk9FUTS...|GdzUXAqSPvH77jgQb...|2016-04-07 01:59:55| false|
 []|--cjT1ICjm_ajiwSK...|jX23x5u_sYz8q2u0S...|2018-12-26 20:36:11| false|
 []|--dhSVoOFDBiMCCwD...|LpGzgTGDr12P_xUaC...|2017-04-23 23:38:33| false|
 []|--dhSVoOFDBiMCCwD...|H9pG0BNhqID2MTj8K...|2016-01-17 01:14:50| false|
 []|--dt6KqbqXc3cIdPN...|flE7Sn2JGs0n3SAta...|2016-01-13 03:59:35| false|
 []|--dt6KqbqXc3cIdPN...|cs1qJvnWUg_f8g020...|2016-01-13 03:56:27| false|
 []|--dt6KqbqXc3cIdPN...|aTd8upPhfxefiecvT...|2018-07-23 21:26:46| false|
 []|--dt6KqbqXc3cIdPN...|aTd8upPhfxefiecvT...|2018-07-04 02:12:54| false|
 []|--dt6KqbqXc3cIdPN...|P7UEPiLj3guembGsm...|2016-01-13 03:43:13| false|
 []|--fyoNxx_yvjnYOyp...|mBi0tqXwGrivaolQD...|2019-06-09 23:16:04| false|
 []|--fyoNxx_yvjnYOyp...|_XN-GwzZwAyIqLKJs...|2019-08-31 04:59:54| false|
 []|--fyoNxx_yvjnYOyp...|6XDIeXx8GJZcGaUDH...|2019-10-24 20:50:38| false|
 []|--nQ4-F7JINAlPir_...|CohyvVtPhFb5GY-gr...|2016-06-03 20:20:12| false|
 []|--nQ4-F7JINAlPir_...|2HKQSZxgfixCLRZQL...|2016-04-02 03:54:41| false|
 []|--pq6tW0VSU7Z-uVe...|tQH_DyUIWYOHMD7CY...|2018-09-02 00:49:01| false|
 []|--rK8GzcNfz88TpsS...|NEoCDeIo1EqSlHkQa...|2015-09-07 02:06:45| false|
 []|--rpfwtT-_B1gDLne...|f9ZzJOiNeoRyXvmOB...|2018-05-04 01:43:47| false|
 []|--t6W1JHbStaCp5RO...|L-Yj3Y1bYKTSg4uE9...|2018-05-29 12:07:47| false|
 []|--t7r3QUhrP2_Ybma...|CopCF0_RumhBLIIB7...|2018-11-12 00:08:54| false|
 []|--vFnC1rxCHpgCXXQ...|9d7N-I00hu-VkXFry...|2015-02-04 23:26:46| false|
 []|--vFnC1rxCHpgCXXQ...|rxU6LyoSNK6Oc9IFh...|2015-03-28 20:32:42| false|
 []|--vFnC1rxCHpgCXXQ...|iObLvZHRZ0RLTQOu-...|2018-01-04 06:04:54| false|
 []|--vFnC1rxCHpgCXXQ...|4LPrdCK0L7GFF6vLP...|2019-02-10 04:05:04| false|
 []|--yKfHy_LnK0Wscyw...|Y0aKsrJ8wDCjvbC0a...|2018-03-30 18:50:22| false|
 []|--yrdC1dIR6VYsW6k...|1F7SjxEmqhFmDfQgp...|2018-02-11 19:08:34| false|
 []|-03ZA_1HdZidy4lEL...|Tz1Oj-tzK-g4iB-Qq...|2006-04-27 02:13:22| false|
 []|-03ZA_1HdZidy4lEL...|Tz1Oj-tzK-g4iB-Qq...|2011-03-01 00:57:43| false|
 []|-03ZA_1HdZidy4lEL...|XCZQdoMRFnjmmB6xP...|2006-04-27 01:34:38| false|
 []|-05T0q5BxB9g0RCKi...|lKom12WnYEjH5FFem...|2012-05-19 23:53:00| false|
 []|-05T0q5BxB9g0RCKi...|sc__kdcFV4IcNTfBx...|2012-05-19 2

#Identifying Elite Status by Elite History
Now that we've determined the Elite status by seeing if the year that the review was posted matches the year the user was considered Elite, we will determine the Elite Status of a yelp user by using their yelp status history. This will help see if the behavior of a user changes based on their Elite status or not. Will users who were once considered Elite yield the same results?

If a Yelp user was an elite user from 2013 to 2015, we would determine that user as an Elite user, regardless if they kept their status or not. In order to do this, an "If-Then" statement is created to see if the Elite years contain any data for the years. Currently the elite-years column contains arrays of years that a user was considered Elite. If the `elite_years column` contains data, then the `is_elite2` column would populate as `true` (if there is no data, then it will be "false").

In [ ]:
df_user_elite_1.createOrReplaceTempView("user_elite1")
df_user_elite_2 = spark.sql("""
SELECT elite_years, user_id, business_id, date, is_elite1, IF(elite_years[0] <> '', true, false) AS is_elite2
FROM user_elite1
""")
df_user_elite_2.show(100)

+--------------------+--------------------+--------------------+-------------------+---------+---------+
 elite_years| user_id| business_id| date|is_elite1|is_elite2|
+--------------------+--------------------+--------------------+-------------------+---------+---------+
 []|---RfKzBwQ8t3wu-L...|Z3ZSar8IVAR2qIupq...|2018-01-13 04:09:32| false| false|
 []|--5Dnq-kuMq5bqvlg...|YYD8tl3ODut_D_sqd...|2014-07-23 02:57:42| false| false|
 []|--DvZltAxZ1sjFn6m...|2t-a8qkgoRiiohThD...|2019-02-02 22:51:46| false| false|
 []|--E5NqaDhTIANG0mX...|JY4kqpElqfflV51PG...|2019-05-22 20:03:46| false| false|
 []|--FoFf79nPpKASxw0...|qNYLt8zizn--wEpT1...|2019-05-06 18:29:22| false| false|
 []|--HeZYR8uZadRXLAQ...|_ZfjpSEO5ntk-1hbn...|2016-04-02 18:03:34| false| false|
 []|--HeZYR8uZadRXLAQ...|ebTvBxSStI9Vf5Tpu...|2016-09-13 13:21:46| false| false|
 []|--I4wRDhmM2J2VLzK...|JmI9nslLD7KZqRr__...|2014-09-14 23:33:43| false| false|
 []|--Oi-hOzz5vCLn9l6...|KligxMWsVPp7Jr_YZ...|2015-04-17 16:32:28| false| false|
 []|--PJ1FuEoTEo-3Cxf...|RESDUcs7fIiihp38-...|2016-08-11 20:51:16| false| false|
 []|--PJ1FuEoTEo-3Cxf...|2weQS-RnoOBhb1KsH...|2017-01-08 21:22:47| false| false|
 []|--RYvmB6UYRyZQqXk...|ZMbY3xmTJ3EB2f1ZF...|2011-08-29 16:41:33| false| false|
 []|--RYvmB6UYRyZQqXk...|MsUI2BgJjptqa1YEA...|2013-11-02 15:24:29| false| false|
 []|--RYvmB6UYRyZQqXk...|i7WdEjPrZkzo7whKd...|2013-01-10 02:50:28| false| false|
 []|--XsqVlXNtt20d5zS...|wk6aHP-vxv9dhmmJV...|2014-07-01 04:12:22| false| false|
 []|--XsqVlXNtt20d5zS...|3enxXjE0vtVaZ-un4...|2014-06-14 01:20:16| false| false|
 []|--XsqVlXNtt20d5zS...|zGXO7B16FXxjc_P0r...|2014-07-01 04:08:19| false| false|
 []|--YhC3mgAU7iexygW...|Xcal78l5dziYYUOfL...|2016-03-15 23:16:11| false| false|
 []|--agAy0vRYwG6WqbI...|ATeOjZzE3VOI-RW7F...|2013-11-26 22:46:06| false| false|
 []|--agAy0vRYwG6WqbI...|chmsA4G55YHyEMsVp...|2014-12-25 23:30:10| false| false|
 []|--bp0mKBU1gk9FUTS...|as_fyujgsdwtV56n3...|2016-04-07 02:07:56| false| false|
 []|--bp0mKBU1gk9FUTS...|GdzUXAqSPvH77jgQb...|2016-04-07 01:59:55| false| false|
 []|--cjT1ICjm_ajiwSK...|jX23x5u_sYz8q2u0S...|2018-12-26 20:36:11| false| false|
 []|--dhSVoOFDBiMCCwD...|LpGzgTGDr12P_xUaC...|2017-04-23 23:38:33| false| false|
 []|--dhSVoOFDBiMCCwD...|H9pG0BNhqID2MTj8K...|2016-01-17 01:14:50| false| false|
 []|--dt6KqbqXc3cIdPN...|flE7Sn2JGs0n3SAta...|2016-01-13 03:59:35| false| false|
 []|--dt6KqbqXc3cIdPN...|cs1qJvnWUg_f8g020...|2016-01-13 03:56:27| false| false|
 []|--dt6KqbqXc3cIdPN...|aTd8upPhfxefiecvT...|2018-07-23 21:26:46| false| false|
 []|--dt6KqbqXc3cIdPN...|aTd8upPhfxefiecvT...|2018-07-04 02:12:54| false| false|
 []|--dt6KqbqXc3cIdPN...|P7UEPiLj3guembGsm...|2016-01-13 03:43:13| false| false|
 []|--fyoNxx_yvjnYOyp...|mBi0tqXwGrivaolQD...|2019-06-09 23:16:04| false| false|
 []|--fyoNxx_yvjnYOyp...|_XN-GwzZwAyIqLKJs...|2019-08-31 04:59:54| false| false|
 []|--fyoNxx_yvjnYOyp...|6XDIeXx8GJZcGaUDH...|2019-10-24 20:50:38| false| false|
 []|--nQ4-F7JINAlPir_...|CohyvVtPhFb5GY-gr...|2016-06-03 20:20:12| false| false|
 []|--nQ4-F7JINAlPir_...|2HKQSZxgfixCLRZQL...|2016-04-02 03:54:41| false| false|
 []|--pq6tW0VSU7Z-uVe...|tQH_DyUIWYOHMD7CY...|2018-09-02 00:49:01| false| false|
 []|--rK8GzcNfz88TpsS...|NEoCDeIo1EqSlHkQa...|2015-09-07 02:06:45| false| false|
 []|--rpfwtT-_B1gDLne...|f9ZzJOiNeoRyXvmOB...|2018-05-04 01:43:47| false| false|
 []|--t6W1JHbStaCp5RO...|L-Yj3Y1bYKTSg4uE9...|2018-05-29 12:07:47| false| false|
 []|--t7r3QUhrP2_Ybma...|CopCF0_RumhBLIIB7...|2018-11-12 00:08:54| false| false|
 []|--vFnC1rxCHpgCXXQ...|9d7N-I00hu-VkXFry...|2015-02-04 23:26:46| false| false|
 []|--vFnC1rxCHpgCXXQ...|rxU6LyoSNK6Oc9IFh...|2015-03-28 20:32:42| false| false|
 []|--vFnC1rxCHpgCXXQ...|iObLvZHRZ0RLTQOu-...|2018-01-04 06:04:54| false| false|
 []|--vFnC1rxCHpgCXXQ...|4LPrdCK0L7GFF6vLP...|2019-02-10 04:05:04| false| false|
 []|--yKfHy_LnK0Wscyw...|Y0aKsrJ8wDCjvbC0a...|2018-03-30 18:50:22| false| false|
 []|--yrdC1dIR6VYsW6k...|1F7SjxEmqhFmDfQgp...|2018-02-11 19:08:34| false| false|
 [

#Identifying Elite Status Based On First Year Elite
Finally, let's look at another way of checking for elite status. We want to see if In the next cell, we will determine the Elite status starting from the year that the user recieved the status. The user would be considered Elite regardless of whether or not they lose the status afterwards.

Essentially the user is considered elite STARTING from the first year that they were elite. Here we say if the `MIN` year of being an Elite user (aka the first year that a user is elite) is `<=` the year of the review, then they are an elite user.

In [ ]:
df_user_elite_2.createOrReplaceTempView("user_elite_2")
df_user_elite_3 = spark.sql("""
SELECT elite_years, user_id, business_id, date, is_elite1, is_elite2, IF(ARRAY_MIN(elite_years) <= YEAR(date), true, false) AS is_elite3
FROM user_elite_2
GROUP BY business_id, elite_years, user_id, date, is_elite1, is_elite2
""")
df_user_elite_3.show(100)

+--------------------+--------------------+--------------------+-------------------+---------+---------+---------+
 elite_years| user_id| business_id| date|is_elite1|is_elite2|is_elite3|
+--------------------+--------------------+--------------------+-------------------+---------+---------+---------+
 []|---RfKzBwQ8t3wu-L...|Z3ZSar8IVAR2qIupq...|2018-01-13 04:09:32| false| false| false|
 []|--5Dnq-kuMq5bqvlg...|YYD8tl3ODut_D_sqd...|2014-07-23 02:57:42| false| false| false|
 []|--DvZltAxZ1sjFn6m...|2t-a8qkgoRiiohThD...|2019-02-02 22:51:46| false| false| false|
 []|--E5NqaDhTIANG0mX...|JY4kqpElqfflV51PG...|2019-05-22 20:03:46| false| false| false|
 []|--FoFf79nPpKASxw0...|qNYLt8zizn--wEpT1...|2019-05-06 18:29:22| false| false| false|
 []|--HeZYR8uZadRXLAQ...|_ZfjpSEO5ntk-1hbn...|2016-04-02 18:03:34| false| false| false|
 []|--HeZYR8uZadRXLAQ...|ebTvBxSStI9Vf5Tpu...|2016-09-13 13:21:46| false| false| false|
 []|--I4wRDhmM2J2VLzK...|JmI9nslLD7KZqRr__...|2014-09-14 23:33:43| false| false| false|
 []|--Oi-hOzz5vCLn9l6...|KligxMWsVPp7Jr_YZ...|2015-04-17 16:32:28| false| false| false|
 []|--PJ1FuEoTEo-3Cxf...|RESDUcs7fIiihp38-...|2016-08-11 20:51:16| false| false| false|
 []|--PJ1FuEoTEo-3Cxf...|2weQS-RnoOBhb1KsH...|2017-01-08 21:22:47| false| false| false|
 []|--RYvmB6UYRyZQqXk...|ZMbY3xmTJ3EB2f1ZF...|2011-08-29 16:41:33| false| false| false|
 []|--RYvmB6UYRyZQqXk...|MsUI2BgJjptqa1YEA...|2013-11-02 15:24:29| false| false| false|
 []|--RYvmB6UYRyZQqXk...|i7WdEjPrZkzo7whKd...|2013-01-10 02:50:28| false| false| false|
 []|--XsqVlXNtt20d5zS...|wk6aHP-vxv9dhmmJV...|2014-07-01 04:12:22| false| false| false|
 []|--XsqVlXNtt20d5zS...|3enxXjE0vtVaZ-un4...|2014-06-14 01:20:16| false| false| false|
 []|--XsqVlXNtt20d5zS...|zGXO7B16FXxjc_P0r...|2014-07-01 04:08:19| false| false| false|
 []|--YhC3mgAU7iexygW...|Xcal78l5dziYYUOfL...|2016-03-15 23:16:11| false| false| false|
 []|--agAy0vRYwG6WqbI...|ATeOjZzE3VOI-RW7F...|2013-11-26 22:46:06| false| false| false|
 []|--agAy0vRYwG6WqbI...|chmsA4G55YHyEMsVp...|2014-12-25 23:30:10| false| false| false|
 []|--bp0mKBU1gk9FUTS...|as_fyujgsdwtV56n3...|2016-04-07 02:07:56| false| false| false|
 []|--bp0mKBU1gk9FUTS...|GdzUXAqSPvH77jgQb...|2016-04-07 01:59:55| false| false| false|
 []|--cjT1ICjm_ajiwSK...|jX23x5u_sYz8q2u0S...|2018-12-26 20:36:11| false| false| false|
 []|--dhSVoOFDBiMCCwD...|LpGzgTGDr12P_xUaC...|2017-04-23 23:38:33| false| false| false|
 []|--dhSVoOFDBiMCCwD...|H9pG0BNhqID2MTj8K...|2016-01-17 01:14:50| false| false| false|
 []|--dt6KqbqXc3cIdPN...|flE7Sn2JGs0n3SAta...|2016-01-13 03:59:35| false| false| false|
 []|--dt6KqbqXc3cIdPN...|cs1qJvnWUg_f8g020...|2016-01-13 03:56:27| false| false| false|
 []|--dt6KqbqXc3cIdPN...|aTd8upPhfxefiecvT...|2018-07-23 21:26:46| false| false| false|
 []|--dt6KqbqXc3cIdPN...|aTd8upPhfxefiecvT...|2018-07-04 02:12:54| false| false| false|
 []|--dt6KqbqXc3cIdPN...|P7UEPiLj3guembGsm...|2016-01-13 03:43:13| false| false| false|
 []|--fyoNxx_yvjnYOyp...|mBi0tqXwGrivaolQD...|2019-06-09 23:16:04| false| false| false|
 []|--fyoNxx_yvjnYOyp...|_XN-GwzZwAyIqLKJs...|2019-08-31 04:59:54| false| false| false|
 []|--fyoNxx_yvjnYOyp...|6XDIeXx8GJZcGaUDH...|2019-10-24 20:50:38| false| false| false|
 []|--nQ4-F7JINAlPir_...|CohyvVtPhFb5GY-gr...|2016-06-03 20:20:12| false| false| false|
 []|--nQ4-F7JINAlPir_...|2HKQSZxgfixCLRZQL...|2016-04-02 03:54:41| false| false| false|
 []|--pq6tW0VSU7Z-uVe...|tQH_DyUIWYOHMD7CY...|2018-09-02 00:49:01| false| false| false|
 []|--rK8GzcNfz88TpsS...|NEoCDeIo1EqSlHkQa...|2015-09-07 02:06:45| false| false| false|
 []|--rpfwtT-_B1gDLne...|f9ZzJOiNeoRyXvmOB...|2018-05-04 01:43:47| false| false| false|
 []|--t6W1JHbStaCp5RO...|L-Yj3Y1bYKTSg4uE9...|2018-05-29 12:07:47| false| false| false|
 []|--t7r3QUhrP2_Ybma...|CopCF0_RumhBLIIB7...|2018-11-12 00:08:54| false| false| false|
 []|--vFnC1rxCHpgCXXQ...|9d7N-I00hu-VkXFry...|2015-02-04 23:26:46| false| false| false|
 []|--vFnC1rxCHpgCXXQ...|rxU6LyoSNK6Oc9IFh...|2015-03-28 20:32:42| false| false| false|
 [

In [ ]:
#This cell creates temp views to prepare for the next cell's join function
df_business_summary.createOrReplaceTempView("business_summary")
df_user_elite_3.createOrReplaceTempView("user_elite_3")

#Identifying the Elite Statuses for the Business in Each Review
Now that we have the three methods of determining Elite status with the columns `is_elite1`, `is_elite2`, and `is_elite3`, we will need to match the businesses to the reviews. We want to identify the Elite Statuses (using the three different methods we’ve created) for the businesses in each of the reviews. This next join matches the businesses with the reviews/reviewers (which was joined from all the previous dataframes) and will include the column, `review_count`. We will need each businesses’ review count to understand the total review count that each business has.

In [ ]:
#This cell joins the user_elite_3 dataframe with the business_summary dataframe through the "business_id" column

df_business_reviews = spark.sql("""
SELECT ue.date, ue.elite_years, bs.review_count, ue.user_id, bs.business_id, ue.is_elite1, ue.is_elite2, ue.is_elite3
FROM business_summary AS bs INNER JOIN user_elite_3 AS ue
ON bs.business_id = ue.business_id
ORDER BY business_id
""")
df_business_reviews.show(500,truncate=True)
df_business_reviews.count()

+-------------------+--------------------+------------+--------------------+--------------------+---------+---------+---------+
 date| elite_years|review_count| user_id| business_id|is_elite1|is_elite2|is_elite3|
+-------------------+--------------------+------------+--------------------+--------------------+---------+---------+---------+
2016-04-02 16:46:14| [2016]| 55|p_fas65QL25jqfV2t...|--6MefnULPED_I942...| true| true| true|
2017-07-30 00:14:21| []| 55|x-Gbs8sVid3yhJIoH...|--6MefnULPED_I942...| false| false| false|
2018-05-21 05:09:07| [2016, 2017, 2018]| 55|EisUuXVeVJN_FcFiE...|--6MefnULPED_I942...| true| true| true|
2017-11-01 19:21:06| []| 55|Z4VF-tv5ibkhv_kzV...|--6MefnULPED_I942...| false| false| false|
2018-09-21 01:07:38| []| 55|yEP9vNFq3edLldNzh...|--6MefnULPED_I942...| false| false| false|
2018-08-19 04:01:19| [2018]| 55|HSHuSCJvIvf_Tof62...|--6MefnULPED_I942...| true| true| true|
2019-03-31 23:23:35| [2018]| 55|s7Ng36gaqCu6nYzUl...|--6MefnULPED_I942...| false| true| true|
2014-11-19 19:12:36| []| 55|ehzvSXsijzgxsYHvX...|--6MefnULPED_I942...| false| false| false|
2015-05-17 21:57:17| []| 55|CogI1DqVfitUCKGNo...|--6MefnULPED_I942...| false| false| false|
2018-10-04 01:37:05| []| 55|NQhvEYuYOa5psBxEo...|--6MefnULPED_I942...| false| false| false|
2017-10-30 04:09:03| []| 55|_c2QPmYaHYrIfLs9D...|--6MefnULPED_I942...| false| false| false|
2015-08-05 20:11:45| [2015, 2016]| 55|8-KBKMJVO-j5YqpHb...|--6MefnULPED_I942...| true| true| true|
2017-08-17 19:21:45| [2017, 2018]| 55|FEg8v92qx3kK4Hu4T...|--6MefnULPED_I942...| true| true| true|
2018-07-08 19:29:51| [2017, 2018]| 55|tKIihU81IA3NjpsAD...|--6MefnULPED_I942...| true| true| true|
2014-01-29 14:38:48| []| 55|h7BVaXP8quy7TlxmH...|--6MefnULPED_I942...| false| false| false|
2016-10-23 00:39:29| [2017, 2018]| 55|MpvqV7lQcl15rflTB...|--6MefnULPED_I942...| false| true| false|
2019-11-02 15:30:58| []| 55|vUchD7H1nGPAZekpM...|--6MefnULPED_I942...| false| false| false|
2017-02-27 18:33:53| [2017, 2018]| 55|tpPg4_1g3QY00L1QJ...|--6MefnULPED_I942...| true| true| true|
2010-12-22 22:33:00|[2009, 2010, 2011...| 55|CxDOIDnH8gp9KXzpB...|--6MefnULPED_I942...| true| true| true|
2019-08-22 15:15:16| [2018]| 55|vX_GHesdmyjEQWTQW...|--6MefnULPED_I942...| false| true| true|
2016-08-25 17:52:54| [2016, 2017, 2018]| 55|V2exkc2JwEdc8UNUN...|--6MefnULPED_I942...| true| true| true|
2018-08-14 21:59:34| []| 55|XNOs3Wz1Q_zdRgm1H...|--6MefnULPED_I942...| false| false| false|
2017-02-14 15:20:00| [2016, 2017, 2018]| 55|dmXpg_unok3cL3eW3...|--6MefnULPED_I942...| true| true| true|
2015-07-16 19:12:31| []| 55|7RabbMUXPXmQt_w_-...|--6MefnULPED_I942...| false| false| false|
2019-11-18 22:29:13| [2016, 2017, 2018]| 55|3OJEa4qXzjSOkD_Jq...|--6MefnULPED_I942...| false| true| true|
2013-09-17 17:45:26| [2018]| 55|HSHuSCJvIvf_Tof62...|--6MefnULPED_I942...| false| true| false|
2019-08-23 17:38:59| []| 55|Wpmklpkb8pii7K1Ur...|--6MefnULPED_I942...| false| false| false|
2011-05-30 23:16:17|[2012, 2013, 2014...| 55|1kNsEAhGU8d8xugMu...|--6MefnULPED_I942...| false| true| false|
2018-10-21 18:45:39| [2018]| 55|rEsBrt6U7i8O4rC81...|--6MefnULPED_I942...| true| true| true|
2011-01-08 02:52:13| []| 55|mKemb3MpSyUbsVUVS...|--6MefnULPED_I942...| false| false| false|
2015-09-28 23:29:41| [2017, 2018]| 55|mnJvynzRIYptrjyKt...|--6MefnULPED_I942...| false| true| false|
2008-08-07 20:26:22|[2007, 2008, 2009...| 55|DS0Idr2XDhz5e7ZfS...|--6MefnULPED_I942...| true| true| true|
2012-04-15 03:42:29|[2011, 2012, 2013...| 55|Tc3GAQdAfOW542ROd...|--6MefnULPED_I942...| true| true| true|
2019-07-12 19:19:26| []| 55|5CrVQeLzpMvKG1hfF...|--6MefnULPED_I942...| false| false| false|
2019-11-24 23:03:38| []| 55|Gl1sgTq65uFtN0aM9...|--6MefnULPED_I942...| false| false| false|
2017-05-31 22:50:50| [2017, 2018]| 55|HPtjvIrhzAUkKsiVk...|--6MefnULPED_I942...| true| true| true|
2014-08-15 23:24:39| [2017]| 55|mcq1qdkjI7M-E1BXe...|--6MefnULPED_I942...| false| true| false|
2019-12-06 15:05:10| [2018]| 55|I5S5BV-n2XhYFjCLT...|--6MefnULPED_I942...| fal

In [ ]:
#Unpersist dataframes that are no longer used to free up memory
df_business_summary.unpersist()
df_user_elite_1.unpersist()
df_user_elite_2.unpersist()
df_user_elite_3.unpersist()

Out[15]: DataFrame[elite_years: array<string>, user_id: string, business_id: string, date: string, is_elite1: boolean, is_elite2: boolean, is_elite3: boolean]

#Preparing the Dataframe for Visualization
Now that we have our three methods of determining Elite statuses, it's time to put the dataframe together with the three different methods of determining an Elite status. We have included the columns: `user_id`, `is_elite1`, `is_elite2`, and `is_elite3`, `business_id`, and `review_count` into one single dataframe. But in order to accurately view our data in chronological order, we have also created a review_order column. The `review_order` column lists the reviews in chronological order by `business_id` and based on the date the review was posted.  The `review_order` column will also help determine the bin number that each review will be categorized in the next few steps.

In [ ]:
df_business_reviews.createOrReplaceTempView("business_reviews")
df_ordered_reviews = spark.sql("""
SELECT review_count, user_id, is_elite1, is_elite2, is_elite3, business_id, date, elite_years, ROW_NUMBER() OVER (PARTITION BY business_id ORDER BY date) AS review_order
FROM business_reviews
""")
df_ordered_reviews.show(truncate=True)

+------------+--------------------+---------+---------+---------+--------------------+-------------------+--------------------+------------+
review_count| user_id|is_elite1|is_elite2|is_elite3| business_id| date| elite_years|review_order|
+------------+--------------------+---------+---------+---------+--------------------+-------------------+--------------------+------------+
 55|DS0Idr2XDhz5e7ZfS...| true| true| true|--6MefnULPED_I942...|2008-08-07 20:26:22|[2007, 2008, 2009...| 1|
 55|cPQZQGC_Rb6-xzKou...| false| true| false|--6MefnULPED_I942...|2010-11-25 19:05:22| [2012]| 2|
 55|j97y1ZAsYABIOcR0K...| false| false| false|--6MefnULPED_I942...|2010-12-06 14:13:14| []| 3|
 55|CxDOIDnH8gp9KXzpB...| true| true| true|--6MefnULPED_I942...|2010-12-22 22:33:00|[2009, 2010, 2011...| 4|
 55|mKemb3MpSyUbsVUVS...| false| false| false|--6MefnULPED_I942...|2011-01-08 02:52:13| []| 5|
 55|1kNsEAhGU8d8xugMu...| false| true| false|--6MefnULPED_I942...|2011-05-30 23:16:17|[2012, 2013, 2014...| 6|
 55|Tc3GAQdAfOW542ROd...| true| true| true|--6MefnULPED_I942...|2012-04-15 03:42:29|[2011, 2012, 2013...| 7|
 55|HSHuSCJvIvf_Tof62...| false| true| false|--6MefnULPED_I942...|2013-09-17 17:45:26| [2018]| 8|
 55|DJTgCoGrU329Z5WZq...| false| false| false|--6MefnULPED_I942...|2013-11-24 22:02:09| []| 9|
 55|h7BVaXP8quy7TlxmH...| false| false| false|--6MefnULPED_I942...|2014-01-29 14:38:48| []| 10|
 55|mcq1qdkjI7M-E1BXe...| false| true| false|--6MefnULPED_I942...|2014-08-15 23:24:39| [2017]| 11|
 55|ehzvSXsijzgxsYHvX...| false| false| false|--6MefnULPED_I942...|2014-11-19 19:12:36| []| 12|
 55|QK_2A9R56Dck-WQ_U...| false| false| false|--6MefnULPED_I942...|2015-01-03 00:32:30| []| 13|
 55|mFwRTTDW0Yr-rFkTF...| true| true| true|--6MefnULPED_I942...|2015-01-04 09:18:58|[2015, 2016, 2017...| 14|
 55|DBHCFW3mSmmOEpONH...| true| true| true|--6MefnULPED_I942...|2015-01-06 21:24:49| [2015, 2016, 2017]| 15|
 55|rY8ia5C7TMsszgZ3J...| false| false| false|--6MefnULPED_I942...|2015-02-22 14:26:55| []| 16|
 55|CogI1DqVfitUCKGNo...| false| false| false|--6MefnULPED_I942...|2015-05-17 21:57:17| []| 17|
 55|7RabbMUXPXmQt_w_-...| false| false| false|--6MefnULPED_I942...|2015-07-16 19:12:31| []| 18|
 55|8-KBKMJVO-j5YqpHb...| true| true| true|--6MefnULPED_I942...|2015-08-05 20:11:45| [2015, 2016]| 19|
 55|mnJvynzRIYptrjyKt...| false| true| false|--6MefnULPED_I942...|2015-09-28 23:29:41| [2017, 2018]| 20|
+------------+--------------------+---------+---------+---------+--------------------+-------------------+--------------------+------------+
only showing top 20 rows

In [ ]:
#Unpersist dataframes that are no longer used to free up memory
df_business_reviews.unpersist()

Out[17]: DataFrame[date: string, elite_years: array<string>, review_count: bigint, user_id: string, business_id: string, is_elite1: boolean, is_elite2: boolean, is_elite3: boolean]

#Excess Reviews

Here, we are filtering out excess reviews (from `ordered_reviews`) per business so that the count of business reviews (in `review_count`) match the quantity of total reviews that a business has. This also means that we are getting rid of reviews past the 100th review, but as we determined earlier via the histogram in the beginning of the notebook, the number of businesses with more than 100 reviews only makes up about 8% of the data, so filtering out reviews past the 100th review wouldn't change any apparent trends in the data for elite reviews.

In [ ]:
df_ordered_reviews.createOrReplaceTempView("ordered_reviews")
df_filtered_reviews = spark.sql("""
SELECT  review_count, user_id, is_elite1, is_elite2, is_elite3, business_id, date, elite_years, review_order
FROM ordered_reviews
WHERE review_order<=review_count
""")
df_filtered_reviews.show()
df_filtered_reviews.count()

+------------+--------------------+---------+---------+---------+--------------------+-------------------+--------------------+------------+
review_count| user_id|is_elite1|is_elite2|is_elite3| business_id| date| elite_years|review_order|
+------------+--------------------+---------+---------+---------+--------------------+-------------------+--------------------+------------+
 55|DS0Idr2XDhz5e7ZfS...| true| true| true|--6MefnULPED_I942...|2008-08-07 20:26:22|[2007, 2008, 2009...| 1|
 55|cPQZQGC_Rb6-xzKou...| false| true| false|--6MefnULPED_I942...|2010-11-25 19:05:22| [2012]| 2|
 55|j97y1ZAsYABIOcR0K...| false| false| false|--6MefnULPED_I942...|2010-12-06 14:13:14| []| 3|
 55|CxDOIDnH8gp9KXzpB...| true| true| true|--6MefnULPED_I942...|2010-12-22 22:33:00|[2009, 2010, 2011...| 4|
 55|mKemb3MpSyUbsVUVS...| false| false| false|--6MefnULPED_I942...|2011-01-08 02:52:13| []| 5|
 55|1kNsEAhGU8d8xugMu...| false| true| false|--6MefnULPED_I942...|2011-05-30 23:16:17|[2012, 2013, 2014...| 6|
 55|Tc3GAQdAfOW542ROd...| true| true| true|--6MefnULPED_I942...|2012-04-15 03:42:29|[2011, 2012, 2013...| 7|
 55|HSHuSCJvIvf_Tof62...| false| true| false|--6MefnULPED_I942...|2013-09-17 17:45:26| [2018]| 8|
 55|DJTgCoGrU329Z5WZq...| false| false| false|--6MefnULPED_I942...|2013-11-24 22:02:09| []| 9|
 55|h7BVaXP8quy7TlxmH...| false| false| false|--6MefnULPED_I942...|2014-01-29 14:38:48| []| 10|
 55|mcq1qdkjI7M-E1BXe...| false| true| false|--6MefnULPED_I942...|2014-08-15 23:24:39| [2017]| 11|
 55|ehzvSXsijzgxsYHvX...| false| false| false|--6MefnULPED_I942...|2014-11-19 19:12:36| []| 12|
 55|QK_2A9R56Dck-WQ_U...| false| false| false|--6MefnULPED_I942...|2015-01-03 00:32:30| []| 13|
 55|mFwRTTDW0Yr-rFkTF...| true| true| true|--6MefnULPED_I942...|2015-01-04 09:18:58|[2015, 2016, 2017...| 14|
 55|DBHCFW3mSmmOEpONH...| true| true| true|--6MefnULPED_I942...|2015-01-06 21:24:49| [2015, 2016, 2017]| 15|
 55|rY8ia5C7TMsszgZ3J...| false| false| false|--6MefnULPED_I942...|2015-02-22 14:26:55| []| 16|
 55|CogI1DqVfitUCKGNo...| false| false| false|--6MefnULPED_I942...|2015-05-17 21:57:17| []| 17|
 55|7RabbMUXPXmQt_w_-...| false| false| false|--6MefnULPED_I942...|2015-07-16 19:12:31| []| 18|
 55|8-KBKMJVO-j5YqpHb...| true| true| true|--6MefnULPED_I942...|2015-08-05 20:11:45| [2015, 2016]| 19|
 55|mnJvynzRIYptrjyKt...| false| true| false|--6MefnULPED_I942...|2015-09-28 23:29:41| [2017, 2018]| 20|
+------------+--------------------+---------+---------+---------+--------------------+-------------------+--------------------+------------+
only showing top 20 rows

Out[18]: 3358707

In [ ]:
#Unpersist dataframes that are no longer used to free up memory
df_ordered_reviews.unpersist()

Out[19]: DataFrame[review_count: bigint, user_id: string, is_elite1: boolean, is_elite2: boolean, is_elite3: boolean, business_id: string, date: string, elite_years: array<string>, review_order: int]

#Determining the Range for Review Counts

In this next cell, we are splitting the range of data into equal sized bins of ten (review_count). We want to visualize the distribution of elite reviews without showing numerous data points of every review that a business has.

In [ ]:
df_filtered_reviews.createOrReplaceTempView("filtered_reviews")
df_ordered_bins = spark.sql("""
SELECT review_count, user_id, business_id, date, is_elite1, is_elite2, is_elite3, elite_years, review_order, 
CASE
  WHEN review_order <= 10 THEN 1
  WHEN review_order <= 20 THEN 2
  WHEN review_order <= 30 THEN 3
  WHEN review_order <= 40 THEN 4
  WHEN review_order <= 50 THEN 5
  WHEN review_order <= 60 THEN 6
  WHEN review_order <= 70 THEN 7
  WHEN review_order <= 80 THEN 8
  WHEN review_order <= 90 THEN 9
  WHEN review_order >= 91 THEN 10
  ELSE -1
END AS bin_number
FROM filtered_reviews
""")
df_ordered_bins.show(truncate=True)

+------------+--------------------+--------------------+-------------------+---------+---------+---------+--------------------+------------+----------+
review_count| user_id| business_id| date|is_elite1|is_elite2|is_elite3| elite_years|review_order|bin_number|
+------------+--------------------+--------------------+-------------------+---------+---------+---------+--------------------+------------+----------+
 55|DS0Idr2XDhz5e7ZfS...|--6MefnULPED_I942...|2008-08-07 20:26:22| true| true| true|[2007, 2008, 2009...| 1| 1|
 55|cPQZQGC_Rb6-xzKou...|--6MefnULPED_I942...|2010-11-25 19:05:22| false| true| false| [2012]| 2| 1|
 55|j97y1ZAsYABIOcR0K...|--6MefnULPED_I942...|2010-12-06 14:13:14| false| false| false| []| 3| 1|
 55|CxDOIDnH8gp9KXzpB...|--6MefnULPED_I942...|2010-12-22 22:33:00| true| true| true|[2009, 2010, 2011...| 4| 1|
 55|mKemb3MpSyUbsVUVS...|--6MefnULPED_I942...|2011-01-08 02:52:13| false| false| false| []| 5| 1|
 55|1kNsEAhGU8d8xugMu...|--6MefnULPED_I942...|2011-05-30 23:16:17| false| true| false|[2012, 2013, 2014...| 6| 1|
 55|Tc3GAQdAfOW542ROd...|--6MefnULPED_I942...|2012-04-15 03:42:29| true| true| true|[2011, 2012, 2013...| 7| 1|
 55|HSHuSCJvIvf_Tof62...|--6MefnULPED_I942...|2013-09-17 17:45:26| false| true| false| [2018]| 8| 1|
 55|DJTgCoGrU329Z5WZq...|--6MefnULPED_I942...|2013-11-24 22:02:09| false| false| false| []| 9| 1|
 55|h7BVaXP8quy7TlxmH...|--6MefnULPED_I942...|2014-01-29 14:38:48| false| false| false| []| 10| 1|
 55|mcq1qdkjI7M-E1BXe...|--6MefnULPED_I942...|2014-08-15 23:24:39| false| true| false| [2017]| 11| 2|
 55|ehzvSXsijzgxsYHvX...|--6MefnULPED_I942...|2014-11-19 19:12:36| false| false| false| []| 12| 2|
 55|QK_2A9R56Dck-WQ_U...|--6MefnULPED_I942...|2015-01-03 00:32:30| false| false| false| []| 13| 2|
 55|mFwRTTDW0Yr-rFkTF...|--6MefnULPED_I942...|2015-01-04 09:18:58| true| true| true|[2015, 2016, 2017...| 14| 2|
 55|DBHCFW3mSmmOEpONH...|--6MefnULPED_I942...|2015-01-06 21:24:49| true| true| true| [2015, 2016, 2017]| 15| 2|
 55|rY8ia5C7TMsszgZ3J...|--6MefnULPED_I942...|2015-02-22 14:26:55| false| false| false| []| 16| 2|
 55|CogI1DqVfitUCKGNo...|--6MefnULPED_I942...|2015-05-17 21:57:17| false| false| false| []| 17| 2|
 55|7RabbMUXPXmQt_w_-...|--6MefnULPED_I942...|2015-07-16 19:12:31| false| false| false| []| 18| 2|
 55|8-KBKMJVO-j5YqpHb...|--6MefnULPED_I942...|2015-08-05 20:11:45| true| true| true| [2015, 2016]| 19| 2|
 55|mnJvynzRIYptrjyKt...|--6MefnULPED_I942...|2015-09-28 23:29:41| false| true| false| [2017, 2018]| 20| 2|
+------------+--------------------+--------------------+-------------------+---------+---------+---------+--------------------+------------+----------+
only showing top 20 rows

In [ ]:
#Unpersist dataframes that are no longer used to free up memory
df_filtered_reviews.unpersist()

Out[21]: DataFrame[review_count: bigint, user_id: string, is_elite1: boolean, is_elite2: boolean, is_elite3: boolean, business_id: string, date: string, elite_years: array<string>, review_order: int]

#Saving Our Findings

In this cell, we are creating our final dataframe before saving it as a table.

In [ ]:
df_ordered_bins.createOrReplaceTempView("ordered_bins")
df_ordered_bins.count()

Out[22]: 3358707

The cell below saves the table in Databricks to be able to send the data to Tableau for visualization.

In [ ]:
df_ordered_bins.write.mode("overwrite").saveAsTable("DF_Final")

##Functions for Visualization

The next 2 cells import functions from the Pillow python library to be able to grab images from our Databricks database and display them as HTML. This will allow us to show the visualizations that have been created in Tableau.

In [ ]:
from PIL import Image
TEMP_DIR = "/temp"

def getWidth(path):
  with Image.open(path) as img:
    width, height = img.size
    return(width)
  
def getDbfsPathName(path):
    # Get the fileinfo containing the path and name
  if path.startswith("/dbfs") != True:
    raise Exception("The path provided does not start with /dbfs")
  new_path = "dbfs:" + path[5:]
  # get the file info for the path
  file_list = dbutils.fs.ls(new_path)
  if len(file_list) != 1:
    raise Exception("The path provided is not a single file on dbfs")
  dbfs_path = file_list[0].path
  filename = file_list[0].name
  return(dbfs_path, filename)
  
def getTempPath(filename):
  # Create the temp directory if it does not exist
  temp_path = "file:" + TEMP_DIR
  dbutils.fs.mkdirs(temp_path)
  temp_list = dbutils.fs.ls(temp_path)
  # get a name to use for the copy
  temp_files = []
  for info in temp_list:
    temp_files.append(info.name)
  increment = 0
  new_name = filename
  while new_name in temp_files:
    increment+=1
    new_name = filename + "." + str(increment)
  access_path = TEMP_DIR + "/" + new_name # used for file opening
  return(access_path)

In [ ]:
import base64
from PIL import Image

def showimage(path, width=0):
  image_string = ""
  img_tag = ""
  dbfs_path, filename = getDbfsPathName(path)
  access_path = getTempPath(filename)
  # copy the file
  copy_path = "file:" + access_path
  dbutils.fs.cp(dbfs_path,copy_path)
  with open(access_path, "rb") as image_file:
    image_string = base64.b64encode(image_file.read() ).decode('utf-8') 
    
  # Is the width setting a positive integer?  A width of 50 means 50%
  if width > 0 and width < 1:
    print("If the width parameter is specified, it must be 1 or more.  A width of 50 means 50%. The width entered was " + str(width) + ", so the original image width was used.")
    width = 0 #reset
    
  if width == 0:
    height = 0
    # Get the width and height of the image in pixels
    with Image.open(access_path) as img:
      width, height = img.size
      
    framewidth = width * 1.1
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  style="width:%ipx;height=%ipx;" /></div>''' % (framewidth,framewidth,image_string, width, height)
  else: # a width was specified
    originalWidth = getWidth(access_path)
    imagewidth = int( width / 100.0 * originalWidth)
    framewidth = int( imagewidth * 1.1 )
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  width="%ipx" height="auto"></div>''' % (framewidth,framewidth,image_string, imagewidth)
  # Clean up the file
  dbutils.fs.rm(copy_path)
  return(img_tag)

##Visualizations for each elite categorization method

All of the visualizations were done in Tableau by putting the `review_order` measure in the row shelf and selecting histogram from the list of options available for the visualizations. For each separate visualization, the `elite_1`, `elite_2`, and `elite_3` was dragged into the color section to display the difference between the amount of elite and non-elite reviews.

In this next cell, we created a histogram of Elite and Non-Elite reviews. We used our first method of elite review categorization, which is to check whether or not the year of the review matches with a year that the user was elite. If it matched, then the review was considered elite, if it didn't match it was considered a non-elite review.

In [ ]:
displayHTML( showimage("/dbfs/FileStore/tables/Elite_1.png", 100) )

This next histogram is created using the second method of elite review categorization, where if a user was ever elite then the review that they created would be considered an elite review.

In [ ]:
displayHTML( showimage("/dbfs/FileStore/tables/Elite_2.png", 100) )

This last histogram was created using the third method of elite review categorization, where starting from the first year that a user was elite, all of their reviews from then on would be considered elite reviews.

In [ ]:
displayHTML( showimage("/dbfs/FileStore/tables/Elite_3.png", 100) )

In all of these histograms we see the same trend of elite to non-elite reviews. The total number of reviews does not start lowering until after 30 as we selected 30 to be the minimum amount of reviews that a business must have before the business is considered within our data. Afterwards, we can see that there are fewer and fewer businesses that have higher total number of reviews. Based on the bars in the histogram, we can see that for all of the elite categorizations, the ratio of elite to non-elite reviews seems to trend downward as businesses get more reviews. In the next section we will be examining exactly what trend in the ratio is and what information we get gain from getting a more precise look at the numbers in the data.

#Getting the Elite:Non-Elite Ratio

The next 9 cells are to get a count of the number of elite and non-elite reivews for each bin based on our three different methods of determining whether or not a user is considered elite. After we get a count of each elite and non-elite reviews, we bring the data into a final table where we can divide the elite reviews by the non-elite reviews to get a ratio of elite to non-elite reviews in a single column. This way, we can see the ratio of elite to non-elite reviewers and the relation between elite reviews and each bin for the reviews.

In [ ]:
#Counting total Elite reviews for the first method of determining Elite reviewers
df_true_elite1_total = spark.sql(""" 
SELECT bin_number, COUNT(is_elite1) AS is_elite1_true
FROM ordered_bins
WHERE is_elite1 = 'true'
GROUP BY bin_number, is_elite1
ORDER BY bin_number
""")
df_true_elite1_total.show()

+----------+--------------+
bin_number|is_elite1_true|
+----------+--------------+
 1| 105852|
 2| 89978|
 3| 83312|
 4| 68634|
 5| 55662|
 6| 45780|
 7| 39203|
 8| 33908|
 9| 29647|
 10| 26388|
+----------+--------------+

In [ ]:
#Counting total Non-Elite reviews for the first method of determining Elite reviewers
df_false_elite1_total = spark.sql(""" 
SELECT bin_number, COUNT(is_elite1) AS is_elite1_false
FROM ordered_bins
WHERE is_elite1 = 'false'
GROUP BY bin_number, is_elite1
ORDER BY bin_number
""")
df_false_elite1_total.show()

+----------+---------------+
bin_number|is_elite1_false|
+----------+---------------+
 1| 383368|
 2| 399242|
 3| 405908|
 4| 360923|
 5| 292392|
 6| 244588|
 7| 209104|
 8| 182347|
 9| 160272|
 10| 142199|
+----------+---------------+

In [ ]:
df_true_elite1_total.createOrReplaceTempView("elite1_true")
df_false_elite1_total.createOrReplaceTempView("elite1_false")
#Joining the Elite/Non-Elite reviews into a single table and getting the ratio for each bin
df_elite1_ratio = spark.sql("""
SELECT e1t.bin_number, e1t.is_elite1_true, e1f.is_elite1_false, e1t.is_elite1_true/e1f.is_elite1_false AS elite1_ratio
FROM elite1_true AS e1t INNER JOIN elite1_false AS e1f
ON e1t.bin_number = e1f.bin_number
ORDER BY bin_number
""")
df_elite1_ratio.show()

+----------+--------------+---------------+-------------------+
bin_number|is_elite1_true|is_elite1_false| elite1_ratio|
+----------+--------------+---------------+-------------------+
 1| 105852| 383368|0.27611068216439555|
 2| 89978| 399242|0.22537208009177392|
 3| 83312| 405908|0.20524847995112194|
 4| 68634| 360923| 0.1901624446211519|
 5| 55662| 292392|0.19036772551916606|
 6| 45780| 244588|0.18717189723126237|
 7| 39203| 209104|0.18748087076287398|
 8| 33908| 182347| 0.1859531552479613|
 9| 29647| 160272|0.18497928521513426|
 10| 26388| 142199| 0.1855709252526389|
+----------+--------------+---------------+-------------------+

In [ ]:
#Unpersist dataframes that are no longer used to free up memory
df_true_elite1_total.unpersist()
df_false_elite1_total.unpersist()

Out[32]: DataFrame[bin_number: int, is_elite1_false: bigint]

In [ ]:
#Counting total Elite reviews for the second method of determining Elite reviewers
df_true_elite2_total = spark.sql(""" 
SELECT bin_number, COUNT(is_elite2) AS is_elite2_true
FROM ordered_bins
WHERE is_elite2 = 'true'
GROUP BY bin_number, is_elite2
ORDER BY bin_number
""")
df_true_elite2_total.show()

+----------+--------------+
bin_number|is_elite2_true|
+----------+--------------+
 1| 145829|
 2| 121605|
 3| 111747|
 4| 94253|
 5| 76421|
 6| 63117|
 7| 53963|
 8| 46862|
 9| 41165|
 10| 36429|
+----------+--------------+

In [ ]:
#Counting total Non-Elite reviews for the second method of determining Elite reviewers
df_false_elite2_total = spark.sql(""" 
SELECT bin_number, COUNT(is_elite2) AS is_elite2_false
FROM ordered_bins
WHERE is_elite2 = 'false'
GROUP BY bin_number, is_elite2
ORDER BY bin_number
""")
df_false_elite2_total.show()

+----------+---------------+
bin_number|is_elite2_false|
+----------+---------------+
 1| 343391|
 2| 367615|
 3| 377473|
 4| 335304|
 5| 271633|
 6| 227251|
 7| 194344|
 8| 169393|
 9| 148754|
 10| 132158|
+----------+---------------+

In [ ]:
df_true_elite2_total.createOrReplaceTempView("elite2_true")
df_false_elite2_total.createOrReplaceTempView("elite2_false")
#Joining the Elite/Non-Elite reviews into a single table and getting the ratio for each bin
df_elite2_ratio = spark.sql("""
SELECT e2t.bin_number, e2t.is_elite2_true, e2f.is_elite2_false, e2t.is_elite2_true/e2f.is_elite2_false AS elite2_ratio
FROM elite2_true AS e2t INNER JOIN elite2_false AS e2f
ON e2t.bin_number = e2f.bin_number
ORDER BY bin_number
""")
df_elite2_ratio.show()

+----------+--------------+---------------+-------------------+
bin_number|is_elite2_true|is_elite2_false| elite2_ratio|
+----------+--------------+---------------+-------------------+
 1| 145829| 343391| 0.4246733315666398|
 2| 121605| 367615| 0.330794445275628|
 3| 111747| 377473| 0.2960397167479528|
 4| 94253| 335304| 0.2810971536277527|
 5| 76421| 271633| 0.2813391598222602|
 6| 63117| 227251|0.27774135207325823|
 7| 53963| 194344| 0.2776674350635986|
 8| 46862| 169393| 0.2766466146771118|
 9| 41165| 148754|0.27673205426408704|
 10| 36429| 132158|0.27564733122474616|
+----------+--------------+---------------+-------------------+

In [ ]:
#Unpersist dataframes that are no longer used to free up memory
df_true_elite2_total.unpersist()
df_false_elite2_total.unpersist()

Out[36]: DataFrame[bin_number: int, is_elite2_false: bigint]

In [ ]:
#Counting total Elite reviews for the third method of determining Elite reviewers
df_true_elite3_total = spark.sql(""" 
SELECT bin_number, COUNT(is_elite2) AS is_elite3_true
FROM ordered_bins
WHERE is_elite3 = 'true'
GROUP BY bin_number, is_elite3
ORDER BY bin_number
""")
df_true_elite3_total.show()

+----------+--------------+
bin_number|is_elite3_true|
+----------+--------------+
 1| 111817|
 2| 96033|
 3| 91161|
 4| 77899|
 5| 63553|
 6| 52390|
 7| 44979|
 8| 39013|
 9| 34295|
 10| 30555|
+----------+--------------+

In [ ]:
#Counting total Non-Elite reviews for the third method of determining Elite reviewers
df_false_elite3_total = spark.sql(""" 
SELECT bin_number, COUNT(is_elite3) AS is_elite3_false
FROM ordered_bins
WHERE is_elite3 = 'false'
GROUP BY bin_number, is_elite3
ORDER BY bin_number
""")
df_false_elite3_total.show()

+----------+---------------+
bin_number|is_elite3_false|
+----------+---------------+
 1| 377403|
 2| 393187|
 3| 398059|
 4| 351658|
 5| 284501|
 6| 237978|
 7| 203328|
 8| 177242|
 9| 155624|
 10| 138032|
+----------+---------------+

In [ ]:
df_true_elite3_total.createOrReplaceTempView("elite3_true")
df_false_elite3_total.createOrReplaceTempView("elite3_false")
#Joining the Elite/Non-Elite reviews into a single table and getting the ratio for each bin
df_elite3_ratio = spark.sql("""
SELECT e3t.bin_number, e3t.is_elite3_true, e3f.is_elite3_false, e3t.is_elite3_true/e3f.is_elite3_false AS elite3_ratio
FROM elite3_true AS e3t INNER JOIN elite3_false AS e3f
ON e3t.bin_number = e3f.bin_number
ORDER BY bin_number
""")
df_elite3_ratio.show()

+----------+--------------+---------------+-------------------+
bin_number|is_elite3_true|is_elite3_false| elite3_ratio|
+----------+--------------+---------------+-------------------+
 1| 111817| 377403| 0.2962801037617613|
 2| 96033| 393187|0.24424256142751413|
 3| 91161| 398059| 0.2290137894131272|
 4| 77899| 351658|0.22151920331685898|
 5| 63553| 284501|0.22338410058312624|
 6| 52390| 237978|0.22014640008740305|
 7| 44979| 203328|0.22121399905571293|
 8| 39013| 177242| 0.2201114859909051|
 9| 34295| 155624|0.22037089394951936|
 10| 30555| 138032|0.22136171322591863|
+----------+--------------+---------------+-------------------+

In [ ]:
#Unpersist dataframes that are no longer used to free up memory
df_true_elite3_total.unpersist()
df_false_elite3_total.unpersist()

Out[40]: DataFrame[bin_number: int, is_elite3_false: bigint]

#Ratio Visualization

To visualize the trend of ratio of elite to non-elite reviews, we put the ratio of elite reviews to each bin of 10 reviews on a line graph for each method of elite categorization.

The cell below shows the trend for the categorization of elite reviews for when the year of the review matches a year that the user was elite.

In [ ]:
df_elite1_ratio.display()

bin_number,is_elite1_true,is_elite1_false,elite1_ratio
1,105852,383368,0.27611068216439555
2,89978,399242,0.22537208009177392
3,83312,405908,0.20524847995112194
4,68634,360923,0.1901624446211519
5,55662,292392,0.19036772551916606
6,45780,244588,0.18717189723126237
7,39203,209104,0.18748087076287398
8,33908,182347,0.1859531552479613
9,29647,160272,0.18497928521513426
10,26388,142199,0.1855709252526389


In the next cell, the line graph shows the trend when reviews are considered elite if the user was ever an elite reviewer.

In [ ]:
df_elite2_ratio.display()

bin_number,is_elite2_true,is_elite2_false,elite2_ratio
1,145829,343391,0.4246733315666398
2,121605,367615,0.330794445275628
3,111747,377473,0.2960397167479528
4,94253,335304,0.2810971536277527
5,76421,271633,0.2813391598222602
6,63117,227251,0.27774135207325823
7,53963,194344,0.2776674350635986
8,46862,169393,0.2766466146771118
9,41165,148754,0.27673205426408704
10,36429,132158,0.27564733122474616


In this last cell the line graph shows the trend when all reviews are considered elite starting from the first year that a user was elite.

In [ ]:
df_elite3_ratio.display()

bin_number,is_elite3_true,is_elite3_false,elite3_ratio
1,111817,377403,0.2962801037617613
2,96033,393187,0.24424256142751413
3,91161,398059,0.2290137894131272
4,77899,351658,0.22151920331685898
5,63553,284501,0.22338410058312624
6,52390,237978,0.22014640008740305
7,44979,203328,0.22121399905571293
8,39013,177242,0.2201114859909051
9,34295,155624,0.22037089394951936
10,30555,138032,0.22136171322591863


In all 3 of these line graphs,we see a sharp decline in the ratio of elite to non-elite reviews until bin 4 (which translates to the 40th review). From bin 4 onward, the ratio of elite to non-elite reviews tends to level out.

#Conclusion

Based on our data and research, it seems that the answer to our initial question of whether or not Elite users tend to review the unreviewed is a resounding "yes". Even when taking into consideration multiple different methods of defining and categorizing what an elite review is, the trends in the data still hold true that elite reviewers tend to be among the first people to review businesses. During the time before 30 reviews, a business can be considered to be unreviewed and the ratio of elite reviews to non-elite reviews is the highest in this time frame. Once a business hits the 40 review mark, the ratio of new elite reviews to non-elite reviews remains steady. This could possibly suggest that non-elite reviewers may take cues to visit unreviewed businesses after seeing that among the few reviews that the businesses do have, a few of them are from elite reviewers.

#Limitations

The minimum amount of reviews was a compromise that we needed to make between having a useful ratio of elite to non-elite reviews and having a larger amount of entries.